In [1]:
import pandas as pd
import numpy as np
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)

pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
import ast
import warnings
import regex as re

warnings.filterwarnings('ignore')
from decimal import *

from enrich_dimensions.rounds import rounds, rounding, re_extract, curate, round_string_float     
from enrich_dimensions.params import parameters, query_from_file
from enrich_dimensions.query_file import query_from_file 
from enrich_dimensions.custom import custom_field 

In [2]:
customer_id = '137'
customer_name='%kimballwest%'
buckets = "Fixings & Fasteners"
# strategy_version_input=705
# attribute_id_input=5131
formatted_attribute = 'head_diameter_metric'

In [3]:
# parameters
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
value='%n/a%'
params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [4]:
import json
dfs = query_from_file(file_name='./query/custom_fields.sql', params=params)
custom_field_df=pd.json_normalize(dfs['custom_fields'])
fields = ['image_url']
df = pd.concat([dfs.drop('custom_fields', axis=1), custom_field_df], axis = 1)
print(len(df))

del df['url']
del df['image_url']
del df['value']
del df['product_id']
del df['bucket_id']

2527


In [5]:
import collections
print([item for item, count in collections.Counter(df['external_id']).items() if count > 1])

[]


In [6]:
pat_mm=rf'''((?<=\d)\s?\-?mm)|()'''         
df['mm_name'] = df['product_name'].apply(lambda x: re_extract(pat_mm, str(x)))
df['mm_ld'] = df['long_desc'].apply(lambda x: re_extract(pat_mm, str(x)))
df['mm_hd'] = df['HeadDiameter'].apply(lambda x: re_extract(pat_mm, str(x)))

pat_hd=r'(?i)(?i)(Head Diameter)|()'
df['head']=df['long_desc'].apply(lambda x: re_extract(pat_hd, x))

# N/A

In [7]:
na=df[(df['mm_ld'].astype(str)=='[]')&(df['mm_name'].astype(str)=='[]')&(df['HeadDiameter'].astype(str)=='nan')]
na[curation_col]='n/a'
matchna=na[['external_id',curation_col]]
print(len(na))

1678


# Curated Fields

In [8]:
print(len(df[(df['HeadDiameter'].astype(str)!='nan')&(df['mm_hd'].astype(str)!='[]')]))
curate=df[(df['HeadDiameter'].astype(str)!='nan')&(df['mm_hd'].astype(str)!='[]')]
curate['hd']=curate['HeadDiameter'].apply(lambda x: f'["{x}"]').apply(lambda x: re.sub(r'(?<=\d)mm',' mm',str(x)))
curate[curation_col]=curate['hd']
matchcurate=curate[['external_id',curation_col]]
from natsort import natsorted
matchcurate[curation_col]=matchcurate[curation_col].str[2:-2].str.split('","').apply(lambda x: '["'+'","'.join(natsorted(x))+'"]')
rounds(matchcurate,curation_col)
matchcurate[curation_col].explode().value_counts()

371


["20 mm"]      75
["13 mm"]      57
["18 mm"]      45
["15 mm"]      34
["16 mm"]      32
["14 mm"]      20
["12 mm"]      15
["10 mm"]      14
["17 mm"]      13
["30 mm"]      10
["22 mm"]       6
["25 mm"]       6
["19 mm"]       6
["9 mm"]        6
["11 mm"]       5
["12.5 mm"]     4
["21 mm"]       3
["6.5 mm"]      2
["8 mm"]        2
["9.5 mm"]      2
["17.5 mm"]     1
["8.5 mm"]      1
["28 mm"]       1
["24 mm"]       1
["26 mm"]       1
["13.5 mm"]     1
["7.5 mm"]      1
["38 mm"]       1
["36.5 mm"]     1
["23 mm"]       1
["32 mm"]       1
["11.5 mm"]     1
["16.5 mm"]     1
["6.3 mm"]      1
Name: Q:head_diameter_metric, dtype: int64

# Name/Description

In [9]:
pat_hd=r'''(?i)(Head Diameter\:\s?\d+\.?\d+\s?mm)|(Head Diameter.{0,3}(?:\d+(?:(?:\-?\s?\d+\/\d+)?(?:\.\d+)?)?|\d+\/\d+)(?:(?:")?(?:”)?(?:'{0,2})?)?\s?\(\d+\.?\d+\s?mm\))|()'''
df['hd_ld']=df['long_desc'].apply(lambda x: re_extract(pat_hd, x))

ex1=df[(df['mm_ld'].astype(str)!='[]')|(df['mm_name'].astype(str)!='[]')|(df['mm_hd'].astype(str)!='[]')]
ex=ex1[(ex1['HeadDiameter'].astype(str)=='nan')&(ex1['hd_ld'].astype(str)!='[]')]
print(len(ex))

ex['hd_ld']=ex['hd_ld'].apply(lambda x: re.sub(r'(?i)(Head Diameter\:\s?)|((?<=\,)\s)|(\))','',str(x))).apply(lambda x: re.sub(r'(.*\()','[',str(x))).apply(lambda x: re.sub(r'(?i)(?<=\d)mm',' mm',str(x))).apply(lambda x: re.sub(r"(?i)'",'"',str(x)))
ex['hd_ld'].explode().value_counts()
ex[curation_col]=ex['hd_ld']
matchex=ex[['external_id',curation_col]]
from natsort import natsorted
matchex[curation_col]=matchex[curation_col].str[2:-2].str.split('","').apply(lambda x: '["'+'","'.join(natsorted(x))+'"]')
rounds(matchex,curation_col)
matchex[curation_col].explode().value_counts()

65


["18 mm"]                    9
["13 mm"]                    7
["15 mm"]                    6
["14 mm"]                    4
["1.5 mm"]                   4
["25 mm"]                    3
["17 mm"]                    3
["19 mm"]                    3
["22 mm"]                    2
["20 mm"]                    2
["23 mm"]                    2
["4.28 mm"]                  2
["12 mm"]                    2
["16 mm","18 mm"]            1
["19.5 mm"]                  1
["16.5 mm","18 mm"]          1
["35 mm"]                    1
["18 mm","23 mm"]            1
["31 mm"]                    1
["15 mm","18 mm"]            1
["16 mm","18 mm","22 mm"]    1
["20 mm","31.5 mm"]          1
["19 mm","20 mm"]            1
["26 mm"]                    1
["10 mm"]                    1
["34 mm"]                    1
["16 mm"]                    1
["13 mm","16 mm"]            1
["5 mm"]                     1
Name: Q:head_diameter_metric, dtype: int64

# Final N/A

In [10]:
final=ex1[(ex1['HeadDiameter'].astype(str)=='nan')&(ex1['hd_ld'].astype(str)=='[]')]
print(len(final))

print(len(final[final['head'].astype(str)=='[]']))
final=final[final['head'].astype(str)=='[]']
final[curation_col]='n/a'
matchfinal=final[['external_id',curation_col]]

412
411


In [11]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchcurate))

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchna))


def get_df_name(df):
    name =[x for x in gglobals() if globals()[x] is df][0]
    return name
print(get_df_name(matchfinal))

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchex))

matchcurate
matchna
matchfinal
matchex


In [12]:
stop

NameError: name 'stop' is not defined

# send to the folder for upload

In [13]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/Kimball MidWest - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('Kimball MidWest', attribute, df, matchcurate) 

In [14]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/Kimball MidWest - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('Kimball MidWest', attribute, df, matchfinal) 

In [15]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/Kimball MidWest - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('Kimball MidWest', attribute, df, matchna) 

In [16]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/Kimball MidWest - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('Kimball MidWest', attribute, df, matchex) 